# Création d'un réseau à partir de l'API Twitter

Sources qui ont servi d'inspiration pour le code pour collecter les données

- https://gist.github.com/PandaWhoCodes/46f58fdead71f4c71453d9ed1e21adf8

Chargement des bibliothèques

In [1]:
import pandas as pd
import tweepy as tw
import json
import time

# Chargement du token d'accès
with open("twitter.keys","r") as f:
    codes = json.load(f)

Définition des variables

In [2]:
liste = pd.read_csv("./base id-usernames.csv",sep=";") # liste de comptes
liste = list(liste["username"])
print(len(liste))

263


Boucle sur l'ensemble des comptes

- un problème rencontré sur un compte, exclu pour le moment, et modification du script avec gestion des exceptions

In [ ]:
auth = tw.OAuthHandler(codes["consumer_key"], codes["consumer_secret"])
auth.set_access_token(codes["access_token"], codes["access_token_secret"])

api = tw.API(auth)

#corpus_followers = {}

# Boucle sur les utilisateurs
for i in liste[71:]: 
    print(i)
    # Boucle pour un compte
    followers = []
    friends = []
    
    # followers
    try:
        for page in tw.Cursor(api.followers, screen_name=i, wait_on_rate_limit=True,count=200).pages():
            try:
                followers.extend(page)
            except tw.TweepError as e:
                print("Going to sleep:", e)
                time.sleep(60)
        print("followers....done")
    except:
        print("Problème followers")
        time.sleep(60)
        
    # friends
    try:
        for page in tw.Cursor(api.friends, screen_name=i, wait_on_rate_limit=True,count=200).pages():
            try:
                friends.extend(page)
            except tw.TweepError as e:
                print("Going to sleep:", e)
                time.sleep(60)
        print("friends....done")
    except:
        print("Problème friends")
        time.sleep(60)
        
            
    # Ajout au corpus
    corpus_followers[i] = {"followers":[d._json for d in followers],"friends":[d._json for d in friends]}
    time.sleep(60)
    
    # Sauvegarde progressive des données
    with open("data.json","w") as f:
        json.dump(corpus_followers,f)

## Chargement des données sauvegardées pour traitement

Attention : le fichier collecté est très lourd. Penser à mettre en forme dès le début le JSON plutôt que tout récupérer ...

In [2]:
with open("data.json","r") as f:
    c = json.load(f)

In [52]:
len(c)

263

Exemple de données

In [3]:
c["noe_kabouche"]["friends"][0]["screen_name"]

'HenriBergeronSP'

Extraction du corpus uniquement des noms

In [5]:
corpus_names_only = {}
for i in c:
    corpus_names_only[i] = {}
    corpus_names_only[i]["friends"] = [j["screen_name"] for j in c[i]["friends"]]
    corpus_names_only[i]["followers"] = [j["screen_name"] for j in c[i]["followers"]]

Création du réseau

In [51]:
import networkx as nx

# Graph dirigé
G = nx.DiGraph()

# Ajout des noeuds de la liste
for n in corpus_names_only:
    G.add_node(n,label = n,friends=len(corpus_names_only[n]["friends"]),
               followers=len(corpus_names_only[n]["followers"]),
               categorie="liste",weight=1)

# Ajout des liens et des noeuds des autres comptes
for n in corpus_names_only:    
    
    # Ajout des friends
    for f in corpus_names_only[n]["friends"]:
        if not G.has_node(f):
            G.add_node(f,label=f,weight=1,categorie="supp")
        else:
            G.nodes[f]["weight"]+=1
       
        if not G.has_edge(n,f):
            G.add_edge(n,f,weight=1,categorie="friends")  
            
    # Ajout des followers
    for f in corpus_names_only[n]["followers"]:
        if not G.has_node(f):
            G.add_node(f,label = f,weight=1,categorie="supp")
        else:
            G.nodes[f]["weight"]+=1
       
        if not G.has_edge(n,f):
            G.add_edge(f,n,weight=1,categorie="followers")   

In [40]:
len(G.edges)

680804

Filtrage pour faire un réseau sans les noeuds de 1 afin de réduire la taille, mais ça reste gros

In [34]:
G_filtre = G.copy()
for i in G.nodes:
    if G.nodes[i]["weight"]<2 and G.nodes[i]["categorie"]!="liste":
        G_filtre.remove_node(i)

Réseau uniquement avec les comptes de la liste initiale

In [52]:
G_liste = G.copy()
for i in G.nodes:
    if G.nodes[i]["categorie"]!="liste":
        G_liste.remove_node(i)

In [44]:
len(G_liste.nodes)

263

Sauvegarder les réseaux das un format compatible Gephi

In [41]:
nx.write_graphml(G_filtre,"reseau_twitter.graphml")

In [53]:
nx.write_graphml(G_liste,"reseau_twitter_liste.graphml")